# **Dataset non bilanciato (Italiano, inglese, spagnolo, tedesco, olandese, russo) con o senza giapponese**

In [ ]:
"""
Qui abbiamo il montaggio del drive
"""

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
"""
Qui abbiamo l'operazione di caricamento dal drive dei dati da 12 landmark
"""

import dill 

x_test_12_0 = dill.load(open("/content/drive/MyDrive/Land_12/unbalanced/x_test_12_0", "rb"))
y_test_12 = dill.load(open("/content/drive/MyDrive/Land_12/unbalanced/y_test_12", "rb"))

x_train_12_0 = dill.load(open("/content/drive/MyDrive/Land_12/unbalanced/x_train_12_0", "rb"))
y_train_12 = dill.load(open("/content/drive/MyDrive/Land_12/unbalanced/y_train_12", "rb"))

In [ ]:
"""
Qui abbiamo l'operazione di caricamento dal drive dei dati da 8 landmark
"""

import dill 

x_test_8_0 = dill.load(open("/content/drive/MyDrive/Land_8/unbalanced_7_languages/x_test_8_0", "rb"))
y_test_8 = dill.load(open("/content/drive/MyDrive/Land_8/unbalanced_7_languages/y_test_8", "rb"))

x_train_8_0 = dill.load(open("/content/drive/MyDrive/Land_8/unbalanced_7_languages/x_train_8_0", "rb"))
y_train_8 = dill.load(open("/content/drive/MyDrive/Land_8/unbalanced_7_languages/y_train_8", "rb"))

In [ ]:
x_train_8_0.shape

(709, 330, 28)

In [ ]:
x_test_8_0.shape

(37, 330, 28)

# **Rete neurale monodirezionale con GRU e LSTM alternati**

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers

path = "/content/drive/MyDrive/uni_unbalanced_best_8_7"
callback_check = ModelCheckpoint(path, monitor="val_accuracy", 
                                 save_best_only=True)
callbacks = [callback_check]

model = Sequential()

model.add(LSTM(128, input_shape=(x_train_8_0.shape[1:]), 
                return_sequences=True, 
                bias_initializer="glorot_uniform"))
model.add(GRU(128, return_sequences=True, activity_regularizer=regularizers.l2(1e-5)))
model.add(LSTM(128, return_sequences=True, dropout=0.1))
model.add(GRU(128, return_sequences=True, activity_regularizer=regularizers.l2(1e-5)))
model.add(LSTM(128, return_sequences=True, dropout=0.1))
model.add(GRU(128, return_sequences=True, activity_regularizer=regularizers.l2(1e-5)))
model.add(LSTM(128, return_sequences=True, dropout=0.1))
model.add(GRU(128, return_sequences=True, activity_regularizer=regularizers.l2(1e-5)))
model.add(LSTM(128, return_sequences=True, dropout=0.1))
model.add(GRU(128, return_sequences=True, activity_regularizer=regularizers.l2(1e-5)))
model.add(LSTM(128, return_sequences=True, dropout=0.1))
model.add(GRU(128, return_sequences=True, activity_regularizer=regularizers.l2(1e-5)))
model.add(LSTM(128))

model.add(Dense(32, bias_initializer="glorot_uniform"))
model.add(Dense(7, activation="softmax"))

opt = tf.keras.optimizers.Adamax(0.0002)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])

model.fit(x_train_8_0, y_train_8, batch_size=64, shuffle=True, epochs=400, 
          callbacks=callbacks, validation_data=(x_test_8_0, y_test_8))

# **Predizioni e analisi dei risultati**

In [ ]:
from tensorflow.keras import models

model_one = models.load_model("/content/drive/MyDrive/uni_unbalanced_best_8_7")

In [ ]:
label = ["italiano", "inglese", "tedesco", "spagnolo", "olandese", "russo", 
         "giapponese"]

In [ ]:
model_one.evaluate(x_test_8_0, y_test_8)

2/2 [==============================] - 3s 109ms/step - loss: 2.9093 - accuracy: 0.6216


[2.909278631210327, 0.6216216087341309]

In [ ]:
import numpy as np
predictions = model_one.predict(x_test_8_0)
max_predictions = []

corrette_lingua = {"italiano": 0, "inglese": 0, "tedesco": 0, "spagnolo": 0, 
        "olandese": 0, "russo": 0, "giapponese": 0}
totali_lingua = {"italiano": 0, "inglese": 0, "tedesco": 0, "spagnolo": 0, 
        "olandese": 0, "russo": 0, "giapponese": 0}
for p in predictions: 
  max_predictions.append(max(p))

total = 0
correct = 0
for (i, p) in enumerate(predictions): 
  print("Predizione massima", max_predictions[i], 
        "Lingua: ", label[np.argmax(predictions[i])], 
        "Lingua corretta: ", label[y_test_8[i]] + "\n")
  
  valore = totali_lingua.get(label[np.argmax(predictions[i])])
  valore += 1
  totali_lingua[label[np.argmax(predictions[i])]] = valore

  if (label[y_test_8[i]] == label[np.argmax(predictions[i])]): 
    valore1 = corrette_lingua.get(label[y_test_8[i]])
    valore1 += 1
    corrette_lingua[label[y_test_8[i]]] = valore1
  
print("Corrette per linguae: ", corrette_lingua)
print("Totali per lingua: ", totali_lingua)
